In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from ast import literal_eval
try:
    from cfuzzyset import cFuzzySet as FuzzySet
except ImportError:
    from fuzzyset import FuzzySet


pd.options.plotting.backend = "plotly"

In [2]:
pt_df = pd.read_csv("./data/pt_ratings.csv")
salaries_df = pd.read_csv("./data/salaries.csv")
rmp_df = pd.read_csv("./data/rmp_ratings.csv")


In [3]:
# Drop professors w/o any ratings / reviews
pt_df = pt_df.dropna(subset=["average_rating", "reviews"])
# pt_df = pt_df.drop(columns=["slug"])
pt_df = pt_df.drop(10561)

# Join names to help name search in salaries
salaries_df["name"] = salaries_df["employee"].apply(
    lambda x: " ".join(x.split(", ")[::-1])
)
salaries_df = salaries_df.replace("\n", " ")
salaries_df["salary"] = salaries_df["salary"].replace('[\$,]', '', regex=True).astype(float)

# Join names to help name search in rmp
def combineNames(row):
    if pd.isnull(row["tMiddlename"]):
        return row["tFname"] + " " + row["tLname"]
    return row["tFname"] + " " + row["tMiddlename"] + " " + row["tLname"]


rmp_df["name"] = rmp_df.apply(combineNames, axis=1)


In [4]:
pt_df.head()

,courses,average_rating,type,reviews,name,slug
6,"['ENME674', 'ENMA300', 'ENME684', 'ENME489Z', ...",5.0000,professor,"[{'professor': 'Abhijit Dasgupta', 'course': '...",Abhijit Dasgupta,dasgupta_abhijit
8,"['ARTH389L', 'ARTH255', 'ARTH768', 'ARTH668A',...",2.8333,professor,"[{'professor': 'Abigail McEwen', 'course': Non...",Abigail McEwen,mcewen
11,"['PHYS405', 'PHYS275', 'PHYS758E', 'PHYS273', ...",4.0000,professor,"[{'professor': 'Abolhassan Jawahery', 'course'...",Abolhassan Jawahery,jawahery
12,"['STAT701', 'STAT700', 'STAT750', 'STAT650', '...",2.7000,professor,"[{'professor': 'Abram Kagan', 'course': 'STAT4...",Abram Kagan,kagan
14,"['ENGL101', 'ENGL243', 'ENGL101S', 'PHSC497', ...",5.0000,professor,"[{'professor': 'Adam Binkley', 'course': 'ENGL...",Adam Binkley,binkley


In [5]:
rmp_df.head()

,tDept,tSid,institution_name,tFname,tMiddlename,tLname,tid,tNumRatings,rating_class,contentType,categoryType,overall_rating,name
0,Computer Science,1270,University of Maryland,Daniel,NaN,Abadi,2361198,1,poor,TEACHER,PROFESSOR,1.0,Daniel Abadi
1,Foreign Languages,1270,University of Maryland,Ali,NaN,Abasi,972417,7,poor,TEACHER,PROFESSOR,2.4,Ali Abasi
2,Physics,1270,University of Maryland,Kevork,NaN,Abazajian,1556681,3,good,TEACHER,PROFESSOR,5.0,Kevork Abazajian
3,Languages,1270,University of Maryland,Ali,NaN,Abbasi,1400739,1,good,TEACHER,PROFESSOR,5.0,Ali Abbasi
4,Economics,1270,University of Maryland,Hossein,NaN,Abbasi,2205764,63,average,TEACHER,PROFESSOR,2.7,Hossein Abbasi


In [6]:
salaries_df.head()

,year,employee,department,division,title,salary,name
0,2013,"Abed, Eyad H",ENGR-Electrical & Computer Engineering,A. James Clark School of Engineering,Prof,216648.00,Eyad H Abed
1,2013,"Abshire, Pamela A.",ENGR-Electrical & Computer Engineering,A. James Clark School of Engineering,Assoc Prof,82872.96,Pamela A. Abshire
2,2013,"Abshire, Pamela A.",ENGR-Institute for Systems Research,A. James Clark School of Engineering,Assoc Prof,55149.36,Pamela A. Abshire
3,2013,"Abts, Leigh R",ENGR-Fischell Department of Bioengineering,A. James Clark School of Engineering,Res Assoc Prof,126334.14,Leigh R Abts
4,2013,"Adams, Douglas J",ENGR-Continuing & Distance Learning in Engr,A. James Clark School of Engineering,Engineer,64260.00,Douglas J Adams


In [7]:
salaries_df.title.value_counts()

Lecturer                        9061
Coordinator                     8767
Prof                            4745
Assoc Prof                      3189
Director                        3079
                                ... 
Prin Agent &\nDirector             1
Extension\nAssociate               1
Prof & Acting Asst\nDean           1
Faculty Extension\nAssistant       1
Associate Program\nDirector        1
Name: title, Length: 822, dtype: int64

In [8]:
pt_df["type"].value_counts()

professor    2852
ta            130
Name: type, dtype: int64

In [9]:
assert len(pt_df) == len(pt_df.drop_duplicates(subset=["name"]))

In [10]:
def get_fl(s):
    sl = s.split()
    return f"{sl[0]} {sl[-1]}"


salary_fuzz = FuzzySet(salaries_df["name"].unique())
salary_fl_fuzz = FuzzySet(salaries_df["name"].apply(get_fl).unique())

matches_salary_to_pt = {}

l = len(pt_df["name"].unique())
for i, pt_name in enumerate(pt_df["name"].unique()):
    matches = salary_fuzz.get(pt_name)
    match_conf, match_name = matches[0]
    print(f"[{i + 1}/{l}] {pt_name} -> {matches}")

    if match_conf <= 0.75:
        matches = salary_fl_fuzz.get(pt_name)
        match_conf, match_name = matches[0]
        print("RETRY")
        print(f"[{i + 1}/{l}] {pt_name} -> {matches}")

    if match_conf >= 0.75:
        matches_salary_to_pt[match_name] = pt_name

len(matches_salary_to_pt)


[1/2982] Abhijit Dasgupta -> [(1, 'Abhijit Dasgupta')]
[2/2982] Abigail McEwen -> [(1, 'Abigail McEwen')]
[3/2982] Abolhassan Jawahery -> [(1, 'Abolhassan Jawahery')]
[4/2982] Abram Kagan -> [(1, 'Abram Kagan')]
[5/2982] Adam Binkley -> [(0.7058823529411764, 'Adam Ross Binkley')]
RETRY
[5/2982] Adam Binkley -> [(1, 'Adam Binkley')]
[6/2982] Adam Hsieh -> [(0.7692307692307692, 'Adam H. Hsieh')]
[7/2982] Adam Porter -> [(0.7857142857142857, 'Adam A. Porter')]
[8/2982] Adi Mahalel -> [(1, 'Adi Mahalel')]
[9/2982] Adil Hassam -> [(1, 'Adil Hassam')]
[10/2982] Adriane Fang -> [(1, 'Adriane Fang')]
[11/2982] Adrianos Papamarcou -> [(1, 'Adrianos Papamarcou')]
[12/2982] Agisilaos Iliadis -> [(1, 'Agisilaos Iliadis')]
[13/2982] Ahmad Karimi-Hakkak -> [(1, 'Ahmad Karimi-Hakkak')]
[14/2982] Ahmet Aydilek -> [(0.8125, 'Ahmet H. Aydilek')]
[15/2982] Ahmet Karamustafa -> [(0.7647058823529411, 'Ali Karamustafa')]
[16/2982] Alan Drew -> [(0.8181818181818181, 'Alan C Drew')]
[17/2982] Alan Kaufman -> 

/tmp/ipykernel_29304/3370154677.py:13: DeprecationWarning: This function will be remove in v3.0.0. Use rapidfuzz.distance.Levenshtein.normalized_similarity instead.
  matches = salary_fuzz.get(pt_name)
/tmp/ipykernel_29304/3370154677.py:18: DeprecationWarning: This function will be remove in v3.0.0. Use rapidfuzz.distance.Levenshtein.normalized_similarity instead.
  matches = salary_fl_fuzz.get(pt_name)


[118/2982] Ben Bezejouh -> [(0.6666666666666667, 'Ben Nkafu Bezejouh')]
RETRY
[118/2982] Ben Bezejouh -> [(1, 'Ben Bezejouh')]
[119/2982] Ben Hurley -> [(0.7692307692307692, 'Ben F. Hurley')]
[120/2982] Ben Shneiderman -> [(0.8333333333333335, 'Ben A. Shneiderman')]
[121/2982] Benjamin Kedem -> [(1, 'Benjamin Kedem')]
[122/2982] Benjamin Rosenthal -> [(0.8571428571428572, 'Benjamin M. Rosenthal')]
[123/2982] Benjamin Shapiro -> [(1, 'Benjamin Shapiro')]
[124/2982] Bernard Cooperman -> [(0.85, 'Bernard D. Cooperman')]
[125/2982] Bertrand Sobesto -> [(1, 'Bertrand Sobesto')]
[126/2982] Beryl Bland -> [(0.6470588235294117, 'Beryl Curry Bland')]
RETRY
[126/2982] Beryl Bland -> [(1, 'Beryl Bland')]
[127/2982] Beth Parent -> [(1, 'Beth Parent')]
[128/2982] Bhanu Telugu -> [(1, 'Bhanu Telugu')]
[129/2982] Bilal Ayyub -> [(0.7857142857142857, 'Bilal M. Ayyub')]
[130/2982] Bo Zhou -> [(1, 'Bo Zhou')]
[131/2982] Bongtae Han -> [(1, 'Bongtae Han')]
[132/2982] Bonnie Dixon -> [(0.7058823529411764,

2509

In [11]:
assert len(set(matches_salary_to_pt.keys())) == len(set(matches_salary_to_pt.values()))

print(len(matches_salary_to_pt))
print(len(salaries_df["name"].unique()))

2509
26529


In [12]:
salaries_df.dtypes

year            int64
employee       object
department     object
division       object
title          object
salary        float64
name           object
dtype: object

In [13]:
tmp = []

for (salary_name, year), salary_rows in salaries_df.groupby(["name", "year"]):
    if year != 2022 or salary_name not in matches_salary_to_pt:
        continue
    
    assert len(salary_rows["name"].unique()) == 1

    # TODO: count only reviews up to year
    # display(rows)

    pt_name = matches_salary_to_pt[salary_name]
    pt_rows = pt_df[pt_df["name"] == pt_name]

    assert len(pt_rows) == 1

    pt_row = pt_rows.iloc[0]

    reviews = literal_eval(pt_row["reviews"])

    tmp.append({
        "name": salary_name,
        "average_rating": pt_row["average_rating"],
        "year": year,
        "salary": salary_rows["salary"].sum(),
        "num_reviews": len(reviews)
    })

tmp = pd.DataFrame(tmp)
tmp

    
    


,name,average_rating,year,salary,num_reviews
0,A W. Kruglanski,2.000,2022,328606.76,1
1,Aaron Finkle,3.000,2022,77699.70,5
2,Aaron Hood,5.000,2022,58556.29,3
3,Abhijit Dasgupta,5.000,2022,227086.06,2
4,Abhinav Bhatele,4.000,2022,180428.27,3
...,...,...,...,...,...
1278,Zhengguo Xiao,4.000,2022,132691.82,1
1279,Zhi Xiang Lin,5.000,2022,10100.00,2
1280,Zhongchi Liu,4.625,2022,128494.72,8
1281,Zohreh Davoudi,2.000,2022,164761.66,4


In [14]:
tmp[tmp["num_reviews"] > 10].sort_values(by=["average_rating", "salary"], ascending=[True, False])

,name,average_rating,year,salary,num_reviews
77,Andris Skuja,1.2857,2022,139087.07,14
432,Heather L Tienson-Tseng,1.4286,2022,57570.00,28
847,Michel Cukier,1.4545,2022,202907.30,11
426,Hanan Samet,1.5455,2022,343420.83,11
699,Leah G Dodson,1.6667,2022,97906.03,57
...,...,...,...,...,...
1171,Thomas E. Murphy,5.0000,2022,244342.68,11
76,Andrew Schonebaum,5.0000,2022,93892.58,11
263,David J. Brookshire,5.0000,2022,50685.84,17
1037,Robert R Lucci,5.0000,2022,50685.84,11


In [15]:
tmp[tmp["num_reviews"] > 10]

,name,average_rating,year,salary,num_reviews
7,Adam A. Porter,2.5455,2022,165991.71,11
12,Adrianos Papamarcou,4.1429,2022,123336.92,28
13,Agisilaos Iliadis,2.3750,2022,163382.57,16
17,Alan C Peel,3.5000,2022,81919.04,12
31,Alexandra E. Bely,4.0000,2022,131174.31,14
...,...,...,...,...,...
1242,Wiseley Wong,4.8750,2022,85330.04,32
1244,Wojciech Czaja,4.0000,2022,211964.19,31
1259,Yavuz A. Oruc,3.1875,2022,112881.19,16
1264,Yijing Wu,3.1818,2022,67632.63,11


In [22]:
tmp[tmp["num_reviews"] > 10].plot(
    kind="scatter",
    x="average_rating",
    y="salary",
    hover_data=['name']
    # line_kws={"color": "red"},
    # fit_reg=True
)

In [ ]:
sns.regplot(
    data=tmp[tmp["num_reviews"] > 10],
    x="average_rating",
    y="salary",
    line_kws={"color": "red"},
    fit_reg=True
)